In [5]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import eval_is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.testing import *
from quantfreedom.plotting.strat_dash import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

prices = pd.read_hdf("../data/bigtest.hd5")

rsi_ind = from_talib(
    func_name='rsi',
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30, 50],
)
entries = eval_is_below(
    want_to_evaluate=rsi_ind,
    user_args=[20,30,40, 50, 60],
    cand_ohlc='close',
)

In [7]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1, 7.1, 1),
    risk_rewards=np.arange(1, 10.1, .5),
    size_pct=1.,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1, 7.1, .5),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1, 7.1, .5),
    tsl_when_pct_from_avg_entry=np.arange(1, 7.1, .5),
    # gains_pct_filter=100,
)

ZeroDivisionError: division by zero

In [6]:
strat_results_df.head(50)

,symbol,entries_col,or_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
11302,BTCUSDT,3,1702,127.00,5.91,22.83,0.85,59.08,"1,059.08"
12102,BTCUSDT,3,2502,129.00,8.20,22.48,0.84,81.95,"1,081.95"
199,BTCUSDT,0,199,59.00,144.07,42.37,0.79,"1,440.72","2,440.72"
12003,BTCUSDT,3,2403,131.00,14.17,22.90,0.78,141.68,"1,141.68"
10415,BTCUSDT,3,815,129.00,4.14,22.48,0.78,41.36,"1,041.36"
10416,BTCUSDT,3,816,129.00,4.14,22.48,0.78,41.36,"1,041.36"
11407,BTCUSDT,3,1807,125.00,4.34,21.60,0.78,43.35,"1,043.35"
12207,BTCUSDT,3,2607,127.00,3.55,21.26,0.78,35.47,"1,035.47"
11217,BTCUSDT,3,1617,129.00,9.35,22.48,0.77,93.48,"1,093.48"
11215,BTCUSDT,3,1615,129.00,10.59,22.48,0.77,105.89,"1,105.89"


In [4]:
settings_results_df.iloc[:,1251]

symbol                           BTCUSDT
entries_col                            5
max_equity_risk_pct                 5.00
risk_rewards                        2.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       5.00
tsl_trail_by_pct                    3.00
tsl_when_pct_from_avg_entry         3.00
Name: 1251, dtype: object

In [5]:
order_records = simulate_up_to_6_nb(
    prices=prices['BTCUSDT'].values,
    entries=entries.iloc[:,[5]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=5,
    risk_rewards=2,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_pcts_init=5,
    tsl_trail_by_pct=3,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [6]:
entries[[("BTCUSDT", 30, 50)]]

symbol,BTCUSDT
RSI_timeperiod,30
RSI_is_below,50
open_time,
2023-03-01 00:00:00,False
2023-03-01 00:05:00,False
2023-03-01 00:10:00,False
2023-03-01 00:15:00,False
2023-03-01 00:20:00,False
...,...
2023-03-15 23:40:00,True


In [7]:
rsi_ind[[('BTCUSDT', 30)]]

symbol,BTCUSDT
RSI_timeperiod,30
open_time,
2023-03-01 00:00:00,NaN
2023-03-01 00:05:00,NaN
2023-03-01 00:10:00,NaN
2023-03-01 00:15:00,NaN
2023-03-01 00:20:00,NaN
...,...
2023-03-15 23:40:00,46.07
2023-03-15 23:45:00,46.59


In [8]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('BTCUSDT', 30)]],
        "entries1": entries[[("BTCUSDT", 30, 50)]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices.BTCUSDT,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
